# Goal:  Ask a Manager survey to analyze different trends in the data.



In [128]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 
import warnings
warnings.filterwarnings("ignore")


In [129]:
#load the data
df = pd.read_csv('Ask A Manager Salary Survey 2023.csv')
print(df.columns)


Index(['Timestamp', 'How old are you?', 'Industry', 'Functional area of job',
       'Job title', 'Job title - additional context', 'Annual salary (gross)',
       'Additional monetary compensation', 'Currency', 'Currency - other',
       'Income - additional context', 'Country', 'State', 'City',
       'Remote or on-site?', 'Years of experience, overall',
       'Years of experience in field', 'Highest level of education completed',
       'Gender', 'Race'],
      dtype='object')


In [130]:
df.head(2)

,Timestamp,How old are you?,Industry,Functional area of job,Job title,Job title - additional context,Annual salary (gross),Additional monetary compensation,Currency,Currency - other,Income - additional context,Country,State,City,Remote or on-site?,"Years of experience, overall",Years of experience in field,Highest level of education completed,Gender,Race
0,4/11/2023 11:02:00,35-44,Government & Public Administration,Engineering or Manufacturing,Materials Engineer,NaN,125000,800.0,USD,NaN,NaN,United States,California,Ridgecrest,On-site,11-20 years,11-20 years,College degree,Man,White
1,4/11/2023 11:02:07,25-34,"Galleries, Libraries, Archives & Museums","Galleries, Libraries, Archives & Museums",Assistant Branch Manager,NaN,71000,0.0,USD,NaN,NaN,United States,Virginia,Fairfax County,On-site,8-10 years,5-7 years,Master's degree,Man,White


In [131]:
#convert to lower case to get unique records
df['job_title'] = df['Job title'].str.lower()
df['education'] = df['Highest level of education completed'].str.lower()


#standardize values in job title by removing white space/ renaming senior

df['job_title'] = df['job_title'].str.strip()

title_remapping={'sr':'senior', 'sr.': 'senior'}

df['job_title'] = df['job_title'].apply(lambda x: title_remapping.get(x,x))

# create total compenstation column 
df['total_comp'] = df['Annual salary (gross)'] + df['Additional monetary compensation']


Explore the data

In [132]:
print('How many observations?', df.shape[0])

How many observations? 16888


In [133]:
print('How many unique job titles?', df.job_title.nunique())

How many unique job titles? 8819


In [134]:
print('Top 5 most represented countries in percent:')
(df['Country'].value_counts(normalize=True)*100).to_frame().head(5)


Top 5 most represented countries in percent:


,proportion
Country,
United States,82.828044
Canada,6.501658
United Kingdom,5.086452
Australia,1.652061
Germany,0.787541


In [135]:

print('Average salary by country & count of submissions:')
df.groupby('Country').agg({
    'Annual salary (gross)':'mean', 'Timestamp':'count'
    }).sort_values(by='Timestamp', ascending=False).round(0).head(10)


Average salary by country & count of submissions:


,Annual salary (gross),Timestamp
Country,,
United States,101354.0,13988
Canada,90466.0,1098
United Kingdom,54926.0,859
Australia,114679.0,279
Germany,69540.0,133
Ireland,69719.0,73
New Zealand,104053.0,64
Sweden,465980.0,31
Netherlands,70323.0,27


In [136]:
df.groupby('Country')['Timestamp'].count().sort_values(ascending=False)

Country
United States                                                  13988
Canada                                                          1098
United Kingdom                                                   859
Australia                                                        279
Germany                                                          133
                                                               ...  
International- mainly US, Canada, Australia, India, Italy          1
International Company with remote work and travel in the US        1
Indonesia                                                          1
Iceland for a U.S. company                                         1
MENA                                                               1
Name: Timestamp, Length: 119, dtype: int64

In [137]:
df['job_title'].unique()

array(['materials engineer', 'assistant branch manager',
       'director of financial aid', ..., 'dance instructor',
       'wedding photographer', 'powertrain integration engineer'],
      dtype=object)

In [138]:
df['education'].value_counts(dropna=False)

education
college degree                                       7623
master's degree                                      5646
some college                                         1147
phd                                                  1080
professional degree (md, jd, etc.)                    882
                                                     ... 
professional quals                                      1
mba student                                             1
professional degree (mlis) and phd                      1
education specialist (ed.s.                             1
associates degree in social & behavioral sciences       1
Name: count, Length: 178, dtype: int64

# Explore the US market

In [139]:
#create subset of the data
df_usa= df[df['Country']==('United States')]

In [140]:
df.head()

,Timestamp,How old are you?,Industry,Functional area of job,Job title,Job title - additional context,Annual salary (gross),Additional monetary compensation,Currency,Currency - other,...,City,Remote or on-site?,"Years of experience, overall",Years of experience in field,Highest level of education completed,Gender,Race,job_title,education,total_comp
0,4/11/2023 11:02:00,35-44,Government & Public Administration,Engineering or Manufacturing,Materials Engineer,NaN,125000,800.0,USD,NaN,...,Ridgecrest,On-site,11-20 years,11-20 years,College degree,Man,White,materials engineer,college degree,125800.0
1,4/11/2023 11:02:07,25-34,"Galleries, Libraries, Archives & Museums","Galleries, Libraries, Archives & Museums",Assistant Branch Manager,NaN,71000,0.0,USD,NaN,...,Fairfax County,On-site,8-10 years,5-7 years,Master's degree,Man,White,assistant branch manager,master's degree,71000.0
2,4/11/2023 11:02:12,35-44,Education (Higher Education),Education (Higher Education),Director of Financial Aid,NaN,60000,0.0,USD,NaN,...,Anadarko,On-site,21-30 years,11-20 years,College degree,Woman,White,director of financial aid,college degree,60000.0
3,4/11/2023 11:02:15,25-34,Education (Higher Education),Government & Public Administration,Administrative Assistant,NaN,42000,NaN,USD,NaN,...,Richmond,On-site,2-4 years,2-4 years,College degree,Man,White,administrative assistant,college degree,NaN
4,4/11/2023 11:02:25,18-24,"Accounting, Banking & Finance",Administration,Executive Assistant,NaN,65000,0.0,USD,NaN,...,Orem,On-site,2-4 years,2-4 years,Some college,Woman,White,executive assistant,some college,65000.0


In [141]:
#create label to classify data related jobs
df_usa['data_job'] = np.where(df_usa['job_title'].str.contains('data|analytics|machine learning|ml|decision|insights|measurement|intelligence|predictive', regex= True, na=False), 'yes', 'no')
df_usa.loc[df_usa['data_job']=='yes'].head()

,Timestamp,How old are you?,Industry,Functional area of job,Job title,Job title - additional context,Annual salary (gross),Additional monetary compensation,Currency,Currency - other,...,Remote or on-site?,"Years of experience, overall",Years of experience in field,Highest level of education completed,Gender,Race,job_title,education,total_comp,data_job
30,4/11/2023 11:03:33,35-44,Government & Public Administration,Computing or Tech,Senior Data Analyst,NaN,110000,4000.0,USD,NaN,...,Fully remote,8-10 years,8-10 years,Master's degree,Woman,White,senior data analyst,master's degree,114000.0,yes
120,4/11/2023 11:06:11,35-44,Nonprofits,Computing or Tech,Data Analyst,Work for a labor union on contract and politic...,73000,0.0,USD,NaN,...,Hybrid,8-10 years,5-7 years,Master's degree,Man,"Hispanic, Latino, or Spanish origin",data analyst,master's degree,73000.0,yes
140,4/11/2023 11:06:34,25-34,Media & Digital,Computing or Tech,Senior Data Analyst,NaN,150000,15000.0,USD,NaN,...,Fully remote,5-7 years,5-7 years,College degree,Man,White,senior data analyst,college degree,165000.0,yes
159,4/11/2023 11:06:57,35-44,Insurance,"Accounting, Banking & Finance",Insurance Data Analyst,Level II (there are three levels total),68500,0.0,USD,NaN,...,Hybrid,11-20 years,1 year or less,Master's degree,Woman,White,insurance data analyst,master's degree,68500.0,yes
161,4/11/2023 11:06:58,35-44,Nonprofits,Computing or Tech,Database Manager,NaN,70000,0.0,USD,NaN,...,Hybrid,11-20 years,5-7 years,College degree,Woman,White,database manager,college degree,70000.0,yes


In [142]:
# We can see that the data related job has higher total compensation.

df_usa.loc[df_usa['data_job'] =='yes']['total_comp'].mean()


print('Average salary:\n')
print(df_usa.groupby('data_job')[['total_comp', 'Annual salary (gross)', 'Additional monetary compensation']].mean())

Average salary:

             total_comp  Annual salary (gross)  \
data_job                                         
no        115362.649962          101073.534805   
yes       129029.501567          110938.105528   

          Additional monetary compensation  
data_job                                    
no                            11642.196010  
yes                           13467.554859  


In [143]:
# Define conditions for 'data_job'
conditions = ['no', 'yes']  # Add more conditions as needed

# Create an empty dictionary to store results
per_results = {}

# Iterate over conditions
for condition in conditions:
    # Calculate the average total compensation for the current condition
    avg_comp = df_usa.loc[df_usa['data_job'] == condition, 'total_comp'].mean()
    
    # Store the result in the dictionary
    per_results[condition] = avg_comp

# Calculate the lift for each condition compared to the first condition
first_condition = conditions[0]
for condition in conditions[1:]:
    perc_chnge = (per_results[condition] - per_results[first_condition]) / per_results[first_condition]
    print(f"Data-related jobs pays more compared to non-data related jobs by: {perc_chnge:.2%}")

Data-related jobs pays more compared to non-data related jobs by: 11.85%


In [144]:
print('Age representation( in percent)')
(df_usa.groupby('data_job')['How old are you?'].value_counts(normalize=True)*100).to_frame()

Age representation( in percent)


proportion
data_job How old are you?            
no       35-44              41.398087
         25-34              33.760118
         45-54              16.490066
         55-64               5.717439
         18-24               1.935247
         65 or over          0.676968
         under 18            0.022075
yes      25-34              38.944724
         35-44              38.944724
         45-54              14.070352
         55-64               5.778894
         18-24               2.010050
         65 or over          0.251256

In [145]:
print('Industry representation - Top 10, percent:')
(df_usa['Industry'].value_counts(normalize=True)*100).to_frame().head(10)


Industry representation - Top 10, percent:


,proportion
Industry,
Education (Higher Education),11.223465
Computing or Tech,10.656913
Nonprofits,9.366036
Health care,8.139702
Government & Public Administration,7.659208
Engineering or Manufacturing,6.174699
"Accounting, Banking & Finance",5.378657
Education (Primary/Secondary),3.600115
Biotechnology & Pharma,3.377797


In [146]:
print('Job title - Top 10:')
df_usa['job_title'].value_counts().to_frame().head(10)


Job title - Top 10:


,count
job_title,
project manager,156
senior software engineer,127
director,120
program manager,113
software engineer,106
teacher,94
manager,75
assistant professor,73
hr manager,72


In [147]:
print('Remote or on-site? Percent:')
(df_usa.groupby('data_job')['Remote or on-site?'].value_counts(normalize=True)*100).to_frame()

Remote or on-site? Percent:


proportion
data_job Remote or on-site?                
no       Hybrid                   39.375369
         On-site                  31.718842
         Fully remote             26.690786
         Other/it's complicated    2.215003
yes      Fully remote             45.591940
         Hybrid                   43.828715
         On-site                  10.075567
         Other/it's complicated    0.503778

In [148]:
print('Years of experience in field (percent)')
(df_usa.groupby('data_job')['Years of experience in field'].value_counts(normalize=True)*100).to_frame()

Years of experience in field (percent)


proportion
data_job Years of experience in field            
no       11-20 years                    29.190581
         5-7 years                      19.308315
         8-10 years                     18.933039
         2-4 years                      16.276674
         21-30 years                     9.624724
         1 year or less                  4.378219
         31-40 years                     2.023547
         41 years or more                0.264901
yes      11-20 years                    24.120603
         2-4 years                      23.869347
         5-7 years                      22.864322
         8-10 years                     15.829146
         21-30 years                     6.030151
         1 year or less                  5.527638
         31-40 years                     1.507538
         41 years or more                0.251256

In [149]:
print('Highest level of education completed')
(df_usa.groupby('data_job')['Highest level of education completed'].value_counts(normalize=True)*100).to_frame().head(10)


Highest level of education completed


proportion
data_job Highest level of education completed            
no       College degree                         45.766337
         Master's degree                        33.906181
         Some college                            6.896297
         PhD                                     5.775188
         Professional degree (MD, JD, etc.)      5.494911
         High School                             1.349757
         Associates                              0.036879
         EdD                                     0.014751
         Associates                              0.014751
         Associates degree                       0.014751

In [154]:
(df_usa.loc[df_usa['data_job'] =='yes']['Highest level of education completed'].value_counts(normalize=True)*100).to_frame().head(10)

,proportion
Highest level of education completed,
College degree,43.324937
Master's degree,37.027708
PhD,11.586902
Some college,5.289673
"Professional degree (MD, JD, etc.)",1.763224
Associates Degree,0.251889
Paralegal certificate,0.251889
Bachelor's not in this field (but still in STEM),0.251889
BS & a grad certificate,0.251889


In [155]:
print('Gender, percent')
(df_usa.groupby('data_job')['Gender'].value_counts(normalize=True)*100).to_frame()

Gender, percent


proportion
data_job Gender                                                      
no       Woman                                              77.435329
         Man                                                16.991870
         Non-binary                                          3.429416
         Other or prefer not to answer                       0.835181
         Woman, Non-binary                                   0.820399
         Man, Non-binary                                     0.332594
         Man, Other or prefer not to answer                  0.051737
         Woman, Other or prefer not to answer                0.036955
         Non-binary, Other or prefer not to answer           0.022173
         Man, Non-binary, Other or prefer not to answer      0.022173
         Man, Woman                                          0.014782
         Woman, Non-binary, Other or prefer not to answer    0.007391
yes      Woman                                              69.017632
         Man                                                23.677582
         Non-binary                                          4.030227
         Other or prefer not to answer                       1.763224
         Woman, Non-binary                                   1.007557
         Man, Non-binary                                     0.251889
         Man, Other or prefer not to answer                  0.251889

In [156]:
print('Race, percent')
(df_usa['Race'].value_counts(normalize=True)*100).to_frame().head(10)

Race, percent


,proportion
Race,
White,87.335869
Asian or Asian American,2.877233
Black or African American,1.879888
"Hispanic, Latino, or Spanish origin",1.815312
Another option not listed here or prefer not to answer,1.528306
"Hispanic, Latino, or Spanish origin, White",1.262826
"Asian or Asian American, White",1.183899
"Black or African American, White",0.365932
"Native American or Alaska Native, White",0.358757
